In [123]:
from random import *
import numpy as np
from matplotlib.pyplot import *
from math import *

In [124]:
n = 10000
points = np.array([[0,0] for i in range(n)])
points.shape[0]

10000

In [125]:
V = 1000/60 #meters per minute
dt = 1 #min
maxi = 68*3.5
tempo = 0

In [126]:
def check(a):
    for i in a: 
        if i[0] > maxi or i[1] > maxi: return 0
    else: return 1

In [127]:
def check_howmany(a):
    result = 0
    for i in a: 
        if i[0] > maxi or i[1] > maxi: result += 1
    return result/a.shape[0]
    


In [128]:
while check_howmany(points) < 0.05:
    for i in points:
        a = uniform(0, 2*pi)
        i[0] += V*dt*sin(a); i[1] += V*dt*cos(a)
    tempo += 1


In [129]:
tempo

191

In [130]:
(68*sqrt(2)/2./61)*1000

788.2501823063154